In [1]:
import numpy as np
import os
from keras.models import Sequential, Model
from keras.layers import Dense, Input, merge
from keras.layers import Reshape,LeakyReLU,ZeroPadding2D
from keras.layers.core import Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Deconvolution2D
from keras.layers.core import Flatten
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import SGD, Adagrad
from PIL import Image
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.objectives import binary_crossentropy
import tensorflow as tf
#from tqdm import tqdm
import scipy.misc as im
#K.set_image_dim_ordering('th') 
IN_CH = 3

Using TensorFlow backend.


In [2]:
import numpy as np
import glob, pickle
import os, sys
import argparse
import cv2
from keras.models import Sequential, Model
from keras.layers import Dense, Input, merge
from keras.layers import Reshape
from keras.layers.core import Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Deconvolution2D
from keras.layers.core import Flatten
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import SGD, Adagrad
from PIL import Image
from keras import backend as K
from keras.layers.normalization import BatchNormalization
import math
K.set_image_dim_ordering('tf') 

img_rows = 256
img_cols = 256
SHAPE = 256
BATCH = 4
IN_CH = 3
OUT_CH = 3
LAMBDA = 100
NF = 64 # number of filter
BATCH_SIZE = 10


In [6]:
def split_input(img,mode='AtoB'):
    """
    img: an 512x256x3 image
    :return: [input, output]
    """
    input, output = img[:,:img_cols,:], img[:,img_cols:,:]

    if mode == 'BtoA':
        input, output = output, input
    if IN_CH == 1:
        input = cv2.cvtColor(input, cv2.COLOR_RGB2GRAY)
    if OUT_CH == 1:
        output = cv2.cvtColor(output, cv2.COLOR_RGB2GRAY)
    return [input, output]

def get_data(datadir):
    #datadir = args.data
    # assume each image is 512x256 split to left and right
    imgs = glob.glob(os.path.join(datadir, '*.jpg'))
    data_X = np.zeros((len(imgs),img_rows,img_cols,3))
    data_Y = np.zeros((len(imgs),img_rows,img_cols,3))
    i = 0
    for file in imgs:
        img = cv2.imread(file,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (img_cols*2, img_rows)) 
        #print('{} {},{}'.format(i,np.shape(img)[0],np.shape(img)[1]))
        

        X, Y = split_input(img)

        data_X[i,:,:,:] = X
        data_Y[i,:,:,:] = Y
        i = i+1
    return data_X, data_Y

In [7]:
data_x,data_y=get_data('train')

In [17]:
#cv2.imshow(data_x[0])
#cv2.waitKey(0)
print(type(data_x))
print(data_x.shape)
img=data_x[0,:,:,:]
#cv2.imshow(img,'img')
#cv2.waitKey(0)

<class 'numpy.ndarray'>
(50, 256, 256, 3)


In [18]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation, Dropout, Flatten, Dense
from keras.layers.merge import Concatenate
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import plot_model

In [19]:
def generator_model():
    
    input_tensor = Input(shape=(img_rows,img_cols,IN_CH)) # type: Input
    output_ch = 3
    filters=128
    x =                       Conv2D(         filters*1, kernel_size=(4, 4), strides=(2, 2), padding="same")( input_tensor )       ; e1 = x
    x = BatchNormalization()( Conv2D(         filters*2, kernel_size=(4, 4), strides=(2, 2), padding="same")( LeakyReLU(0.2)(x) ) ); e2 = x
    x = BatchNormalization()( Conv2D(         filters*4, kernel_size=(4, 4), strides=(2, 2), padding="same")( LeakyReLU(0.2)(x) ) ); e3 = x
    x = BatchNormalization()( Conv2D(         filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( LeakyReLU(0.2)(x) ) ); e4 = x
    x = BatchNormalization()( Conv2D(         filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( LeakyReLU(0.2)(x) ) ); e5 = x
    x = BatchNormalization()( Conv2D(         filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( LeakyReLU(0.2)(x) ) ); e6 = x
    x = BatchNormalization()( Conv2D(         filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( LeakyReLU(0.2)(x) ) ); e7 = x
    x =                       Conv2D(         filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( LeakyReLU(0.2)(x) )  ; e8 = x
    # dec
    x = BatchNormalization()( Conv2DTranspose(filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) ) ); x = Concatenate()([Dropout(0.5)(x), e7])
    x = BatchNormalization()( Conv2DTranspose(filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) ) ); x = Concatenate()([Dropout(0.5)(x), e6])
    x = BatchNormalization()( Conv2DTranspose(filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) ) ); x = Concatenate()([Dropout(0.5)(x), e5])
    x = BatchNormalization()( Conv2DTranspose(filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) ) ); x = Concatenate()([x, e4])
    x = BatchNormalization()( Conv2DTranspose(filters*4, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) ) ); x = Concatenate()([x, e3])
    x = BatchNormalization()( Conv2DTranspose(filters*2, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) ) ); x = Concatenate()([x, e2])
    x = BatchNormalization()( Conv2DTranspose(filters*1, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) ) ); x = Concatenate()([x, e1])
    x =                       Conv2DTranspose(output_ch, kernel_size=(4, 4), strides=(2, 2), padding="same")( Activation("relu")(x) )
    
    x = Activation("tanh")(x)
    
    unet = Model(inputs=input_tensor, outputs=x)
    
    return unet

In [24]:
gx=generator_model()
gx.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 128, 128, 128)     6272      
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 64, 256)       524544    
_________________________________________________________________
batch_normalization_14 (Batc (None, 64, 64, 256)       1024      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 512)       2097664   
__________

In [21]:
def discriminator_model():    
    inputs = Input(shape=(img_cols,img_rows,IN_CH*2))
    d = ZeroPadding2D(padding=(1,1))(inputs)
    d = Convolution2D(64,4,4,subsample=(2,2))(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = ZeroPadding2D(padding=(1,1))(d)
    d = Convolution2D(128,4,4,subsample=(2,2))(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = ZeroPadding2D(padding=(1,1))(d)
    d = Convolution2D(256,4,4,subsample=(2,2))(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = ZeroPadding2D(padding=(1,1))(d)
    d = Convolution2D(512,4,4,subsample=(1,1))(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = ZeroPadding2D(padding=(1,1))(d)
    d = Convolution2D(1,4,4,subsample=(1,1),activation='sigmoid')(d)
    model = Model(inputs,d)
    return model

In [25]:
dx=discriminator_model()
dx.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2))`
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (4, 4), strides=(2, 2))`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256, 256, 6)       0         
_________________________________________________________________
zero_padding2d_6 (ZeroPaddin (None, 258, 258, 6)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 128, 128, 64)      6208      
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 128, 128, 64)      0         
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 130, 130, 64)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 64, 64, 128)       131200    
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 64, 64, 128)       0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (4, 4), strides=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (4, 4), strides=(1, 1))`
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (4, 4), activation="sigmoid", strides=(1, 1))`


In [23]:
def generator_containing_discriminator(generator, discriminator):
    inputs = Input((img_cols, img_rows,IN_CH))
    x_generator = generator(inputs)
    
    merged = merge([inputs, x_generator], mode='concat',concat_axis=-1)
    discriminator.trainable = False
    x_discriminator = discriminator(merged)
    
    model = Model(inputs,[x_generator,x_discriminator])
    
    return model

In [26]:
gcd=generator_containing_discriminator(gx,dx)
gcd.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
model_3 (Model)              (None, 256, 256, 3)       217648387 
_________________________________________________________________
merge_1 (Merge)              (None, 256, 256, 6)       0         
_________________________________________________________________
model_4 (Model)              (None, 30, 30, 1)         2767809   
Total params: 220,416,196.0
Trainable params: 220,396,484.0
Non-trainable params: 19,712.0
_________________________________________________________________


In [27]:
def discriminator_on_generator_loss(y_true,y_pred):
    return K.mean(K.binary_crossentropy(y_pred,y_true), axis=(1,2,3))

def generator_l1_loss(y_true,y_pred):
    return K.mean(K.abs(y_pred - y_true),axis=(1,2,3))

In [28]:
def generate_pic(generator,target,e):
    pic = generator.predict(target)
    pic = np.squeeze(pic,axis=0)
    target = np.squeeze(target,axis=0)
    im.imsave('target_%d.png' % e,target)
    im.imsave('pic_%d.png' % e,pic)


In [29]:
def train(epochs,batchsize):
    
    
    
    pic,target=get_data('train')
    pic = pic.astype('float32')
    target = target.astype('float32')
    pic = (pic - 127.5) / 127.5
    target = (target - 127.5) / 127.5
    batchCount = pic.shape[0] / batchsize
    print('Epochs  ',epochs)
    print('Batch_size   ',batchsize)
    print('Batches per epoch    ',batchCount)
    
    
    
    
    generator = generator_model()
    
    discriminator = discriminator_model()
    
    gan = generator_containing_discriminator(generator,discriminator)
    
    generator.compile(loss=generator_l1_loss, optimizer='RMSprop')
    
    gan.compile(loss=[generator_l1_loss,discriminator_on_generator_loss] , optimizer='RMSprop')
    
    discriminator.trainable = True
    
    discriminator.compile(loss=discriminator_on_generator_loss, optimizer='RMSprop')
    
    
    G_loss = []
    D_loss = []
    for e in range(1,epochs+1):
        print('Epoch       {}',e)
        for i in range(batchCount):
            random_number = np.random.randint(1,pic.shape[0],size=batchsize)
            batch_pic = pic[random_number]
            batch_target = target[random_number]
            batch_target2 = np.tile(batch_target,(2,1,1,1))
            y_dis = np.zeros((2*batchsize,30,30,1))
            y_dis[:batchsize] = 1.0
            generated_pic = generator.predict(batch_target)
            #Default is concat first dimention
            concat_pic = np.concatenate((batch_pic,generated_pic))
            
            dis_input = np.concatenate((concat_pic,batch_target2),axis=-1)
            
            dloss = discriminator.train_on_batch(dis_input,y_dis)
            
            random_number = np.random.randint(1,pic.shape[0],size=batchsize)
            
            train_target = target[random_number]
            
            batch_pic = pic[random_number]
            
            y_gener = np.ones((batchsize,30,30,1))
            
            discriminator.trainable = False
            
            gloss = gan.train_on_batch(train_target,[batch_pic,y_gener])
            
            discriminator.trainable = True
            
        G_loss.append(gloss)
        D_loss.append(dloss)
        if e % 50 == 0 or e == 1:
            generate_pic(generator,target[0:1],e)
            
            generator.save('Model_para/pix2pix_g_epoch_%d.h5' % e)
            
            discriminator.save('Model_para/pix2pix_d_epoch_%d.h5' % e)
            
            gan.save('Model_para/pix2pix_gan_epoch_%d.h5' % e)
            
    D_loss = np.array(D_loss)
    G_loss = np.array(G_loss)
    np.save('Model_para/dloss.npy',D_loss)
    np.save('Model_para/gloss.npy',G_loss)